In [2]:
import numpy as np
import pandas as pd
from create_model_features import trends_features
from copy import deepcopy
import datetime
################## INSERT CCY You want to RUN
ccy = "EURUSD"
ccy1 = ccy[:3]
ccy2 = ccy[3:]
edi_location = r"C:\Users\edgil\Documents\Masters\dissertation\data\EDI.csv"
esi_location = r"C:\Users\edgil\Documents\Masters\dissertation\data\ESI.csv"
features = ["EDI_"+ccy1 + ccy2 + "_spread", "EDI_G10_normalised", "EDI_Global_normalised",
            "ESI_"+ccy1 + ccy2 + "_spread", "ESI_G10_normalised", "ESI_Global_normalised"]
EDI = pd.read_csv(edi_location)
ESI = pd.read_csv(esi_location)
EDI['Date'] =  pd.to_datetime(EDI['Date'], format= '%d/%m/%Y')
ESI['Date'] =  pd.to_datetime(ESI['Date'], format= '%d/%m/%Y')
EDI['Date'] =  EDI['Date'].dt.date
ESI['Date'] =  ESI['Date'].dt.date
EDI["EDI_"+ ccy1 + ccy2 + "_spread"] = EDI["EDI_" +ccy1 + "_normalised"] - EDI["EDI_" +ccy2 + "_normalised"]
ESI["ESI_"+ ccy1 + ccy2 + "_spread"] = ESI["ESI_" +ccy1 + "_normalised"] - ESI["ESI_" + ccy2 + "_normalised"]
EDI_tradable = EDI[["EDI_"+ccy1 + ccy2 + "_spread", "EDI_G10_normalised", "EDI_Global_normalised"]].cumsum()
ESI_tradable = ESI[["ESI_"+ccy1 + ccy2 + "_spread", "ESI_G10_normalised", "ESI_Global_normalised"]].cumsum()
EDI_tradable['Date'] = EDI['Date']
ESI_tradable['Date'] = ESI['Date']
short, medium, long, longest, medium_multiplier,long_multplier = 21, 55, 100, 200, 1, 1 # as all in days.
for col in features:
    if col in list(EDI_tradable.columns):
        data = deepcopy(EDI_tradable)
        data = trends_features(data,col, short, medium, long,longest,medium_multiplier,long_multplier)
        EDI_tradable[col+"_spotvma"] = data['spot_v_HF']
        EDI_tradable[col+"_madiff"] = data['HF_ema_diff']
    if col in list(ESI_tradable.columns):
        data = deepcopy(ESI_tradable)
        data = trends_features(data,col, short, medium, long, longest,medium_multiplier,long_multplier)
        ESI_tradable[col+"_spotvma"] = data['spot_v_HF']
        ESI_tradable[col+"_madiff"] = data['HF_ema_diff']
print(EDI_tradable.columns)
signal_location = r"C:\Users\edgil\Documents\Masters\dissertation\data\best_results\Signals.csv"
signals = pd.read_csv(signal_location)
signals['Date'] = pd.to_datetime(signals['Date'], format= '%d/%m/%Y %H:%M')
signals['DDMMYY'] = signals['Date'].dt.date # signals['Date'].apply(get_ddmmyy)
#signals['DDMMYY'] = pd.to_datetime(signals['DDMMYY'])
trend_estimation = signals.groupby('DDMMYY').sum().reset_index(drop = False)
# i.e. the average of the ccy kevel across each hour
trend_estimation['CCY'] = signals.groupby('DDMMYY').mean().reset_index(drop = False)['CCY']
# overwriting the dte arg as its easier to understand
trend_estimation['Date'] = trend_estimation['DDMMYY']
trend_estimation[ccy1+ccy2 +"_vol"] = trend_estimation['logret'].rolling(60).std()*np.sqrt(260)
trend_estimation = pd.merge(trend_estimation,EDI_tradable,how="left" ,on= "Date")
trend_estimation = pd.merge(trend_estimation,ESI_tradable,how="left" ,on= "Date")
trend_location = r"C:\Users\edgil\Documents\Masters\dissertation\data\best_results\trendestimation.csv"
trend_estimation.replace(np.nan,0).to_csv(trend_location)

C:\Users\edgil\Documents\Masters\dissertation\code\create_model_features.py:29: RuntimeWarning: divide by zero encountered in log
  data["logret"] = np.log(data[CCY_COL]) - np.log(data[CCY_COL].shift(1))
C:\Users\edgil\Documents\Masters\dissertation\code\create_model_features.py:29: RuntimeWarning: invalid value encountered in log
  data["logret"] = np.log(data[CCY_COL]) - np.log(data[CCY_COL].shift(1))


Index(['EDI_EURUSD_spread', 'EDI_G10_normalised', 'EDI_Global_normalised',
       'Date', 'EDI_EURUSD_spread_spotvma', 'EDI_EURUSD_spread_madiff',
       'EDI_G10_normalised_spotvma', 'EDI_G10_normalised_madiff',
       'EDI_Global_normalised_spotvma', 'EDI_Global_normalised_madiff'],
      dtype='object')


In [31]:
from matplotlib import pyplot as plt
# model features
features = ["EDI_"+ccy1 + ccy2 + "_spread", "EDI_G10_normalised", "EDI_Global_normalised",
            "ESI_"+ccy1 + ccy2 + "_spread", "ESI_G10_normalised", "ESI_Global_normalised"]


array([  0.        ,  -0.39104105,  -0.25810771, ...,  -9.83269382,
       -10.32362597,  -8.81085717])

In [3]:
# next steps--> creating an engine to run a rf on the 1 month head returns of the startegy based on the features
# use var imp type module.
from model_functions import standardise_data
from sklearn.preprocessing import StandardScaler
from model_functions import create_train_test_file
from run_decision_tree import set_params_trend_estimate
trend_estimate_dict = set_params_trend_estimate()
trade_horizon = trend_estimate_dict['trade_horizon'] # roughly 1 month ahead.
std_window = trend_estimate_dict['std_window'] # 1 year
train_size = trend_estimate_dict['train_size']# no. data points in the train
test_split = trend_estimate_dict['test_split']
test_buffer = trend_estimate_dict['test_buffer']
concat_results = False
trend_strats = ["Linear","SVM_erf","RF_erf","LSTM1_erf","LSTM2_erf"]
T_features = ["ESI_EURUSD_spread_spotvma", "ESI_EURUSD_spread_madiff",       "ESI_G10_normalised_spotvma",
                   "ESI_G10_normalised_madiff", "ESI_Global_normalised_spotvma", "ESI_Global_normalised_madiff",
                   "EDI_EURUSD_spread_spotvma", "EDI_EURUSD_spread_madiff",       "EDI_G10_normalised_spotvma",
                   "EDI_G10_normalised_madiff", "EDI_Global_normalised_spotvma",  "EDI_Global_normalised_madiff"]

feats = deepcopy(T_features)
feats.append("Date")
feats.append("SVM_erf")
# now kick off the learning approach
df = trend_estimation[feats]
df['target'] = df["SVM_erf"].iloc[::-1].shift(2).rolling(trade_horizon).sum().values[::-1]
# maybe best to standardise on the skleanr methos
# df = standardise_data(df, feats, T_features , std_window)
data_size = int(df.shape[0]*train_size) # data size implies the size of the training set
train, test = create_train_test_file(df, data_size, test_split, test_buffer, concat_results)

C:\Users\edgil\AppData\Local\Programs\Python\Python36\lib\site-packages\ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [6]:
train_sample = StandardScaler.fit_transform(train[T_features])
train_sample['target'] = train['target']
test_sample = StandardScaler.transform(test[T_features])
test_sample['target'] = test['target']
rf_dict = set_params_random_forests()
ntree = rf_dict['ntree']
max_features = rf_dict['max_features']
decision_tree(train_sample, test_sample,use_classifier, use_risk_adjusted,ntree, max_features)

TypeError: fit_transform() missing 1 required positional argument: 'X'